# Actividad 8

### Analisis y entendimiento del Data Set

In [1]:
# empecemos importando las librerias necesarisas
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator as MCE
from pyspark.ml.classification import DecisionTreeClassifier as DTC
from pyspark.ml.classification import GBTClassifier as GBTC
from pyspark.ml.classification import RandomForestClassifier as RFC

In [2]:
# Ceemos la sesion:
spark = SparkSession.builder.appName('IrisDS').getOrCreate()

21/09/23 23:57:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
#leamos el data set
df = spark.read.csv('Iris_DataSheet.csv', header=True).cache()
# primer vistazo
df.show(10)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
+---+-------------+------------+-------------+------------+-----

In [4]:
# lineas del data set:
df.count()

150

In [5]:
# columnas y su tipo:
df.dtypes

[('Id', 'string'),
 ('SepalLengthCm', 'string'),
 ('SepalWidthCm', 'string'),
 ('PetalLengthCm', 'string'),
 ('PetalWidthCm', 'string'),
 ('Species', 'string')]

In [6]:
# Adecuemos el data set dando el fromato adecuado a las columnas numéricas
df = df.select(
    col('Id').cast('int'),
    col('SepalLengthCm').cast('float'),
    col('SepalWidthCm').cast('float'),
    col('PetalLengthCm').cast('float'),
    col('PetalWidthCm').cast('float'),
    col('Species')
)
df.show(5)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



## Transformar en vector los datos numéricos empleando Vector Assembler

In [7]:
# lo primero, definamos que datos(columnas) se han de combertir en vector:
req_feat = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']

# creemos los vectores
assembler = VectorAssembler(inputCols=req_feat, outputCol='Sizes')

# añadamoslo al dataset
df = assembler.transform(df)

In [8]:
df.show(10)

+---+-------------+------------+-------------+------------+-----------+--------------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|               Sizes|
+---+-------------+------------+-------------+------------+-----------+--------------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[5.09999990463256...|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[4.90000009536743...|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[4.69999980926513...|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.59999990463256...|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0,3.5999999046...|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|[5.40000009536743...|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|[4.59999990463256...|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa

## Transformar los datos correspondientes a la columna especies usando StringIndexer

In [9]:
# convirtamos la columna 'Species' en una columnas numérica
df = StringIndexer(inputCol='Species', outputCol='SpeciesID').fit(df).transform(df)
# A setosa le asignará el 1, a versicolor el 2 y a virginica, el 3

In [10]:
df.show(10)

+---+-------------+------------+-------------+------------+-----------+--------------------+---------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|               Sizes|SpeciesID|
+---+-------------+------------+-------------+------------+-----------+--------------------+---------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[5.09999990463256...|      0.0|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[4.90000009536743...|      0.0|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[4.69999980926513...|      0.0|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.59999990463256...|      0.0|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0,3.5999999046...|      0.0|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|[5.40000009536743...|      0.0|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|[4

In [11]:
#quitemos la columna Species
df.drop('Species')

DataFrame[Id: int, SepalLengthCm: float, SepalWidthCm: float, PetalLengthCm: float, PetalWidthCm: float, Sizes: vector, SpeciesID: double]

**Creemos ahora las particiones Train Test**

In [12]:
#empelaremos el 85% del data set para Train y el 15% para test
(train, test) = df.randomSplit([0.85, 0.15])

## Calcular la predicción y la precisión del modelo empleando Decision Tree Classifier (DTC)

In [13]:
# Antes de nada, creemos el modelo de prediccion
dtc = DTC(labelCol='SpeciesID', featuresCol='Sizes',maxDepth=10)

Aumentando el valor de 'maxDepth' de 5 a 10 la prediccion del modelo mejora, al menos en este caso, de ahi para arriba parece que se crea algo de overfitting

In [14]:
# Entrenemos el modelo:
dtc_model = dtc.fit(train)

In [15]:
# Generemos una prediccion:
dtc_pred = dtc_model.transform(test)
dtc_pred.toPandas().head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Sizes,SpeciesID,rawPrediction,probability,prediction
0,3,4.7,3.2,1.3,0.2,Iris-setosa,"[4.699999809265137, 3.200000047683716, 1.29999...",0.0,"[43.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
1,9,4.4,2.9,1.4,0.2,Iris-setosa,"[4.400000095367432, 2.9000000953674316, 1.3999...",0.0,"[43.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
2,11,5.4,3.7,1.5,0.2,Iris-setosa,"[5.400000095367432, 3.700000047683716, 1.5, 0....",0.0,"[43.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
3,24,5.1,3.3,1.7,0.5,Iris-setosa,"[5.099999904632568, 3.299999952316284, 1.70000...",0.0,"[43.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
4,35,4.9,3.1,1.5,0.1,Iris-setosa,"[4.900000095367432, 3.0999999046325684, 1.5, 0...",0.0,"[43.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0


### Evaluemos el modelo:

In [16]:
# configuremos el evaluador:
evaluator = MCE(labelCol='SpeciesID', predictionCol='prediction', metricName='accuracy')

In [17]:
#  Ahora si, evaluemos el modelo:
accuracy = evaluator.evaluate(dtc_pred)
print(f'DTC test accuracy: {accuracy}')

DTC test accuracy: 0.8888888888888888


## Calcular la predicción y la precisión del modelo empleando Gradient-boosted tree classifier GBTC

**Como el modelo GBTC solo permite trabajar con predicciones binarias, será necesario adecuar los datos:**

In [18]:
# Convirtamos el data frame que tenemos a pandas:
pdf = df.toPandas()
pdf.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Sizes,SpeciesID
0,1,5.1,3.5,1.4,0.2,Iris-setosa,"[5.099999904632568, 3.5, 1.399999976158142, 0....",0.0
1,2,4.9,3.0,1.4,0.2,Iris-setosa,"[4.900000095367432, 3.0, 1.399999976158142, 0....",0.0
2,3,4.7,3.2,1.3,0.2,Iris-setosa,"[4.699999809265137, 3.200000047683716, 1.29999...",0.0
3,4,4.6,3.1,1.5,0.2,Iris-setosa,"[4.599999904632568, 3.0999999046325684, 1.5, 0...",0.0
4,5,5.0,3.6,1.4,0.2,Iris-setosa,"[5.0, 3.5999999046325684, 1.399999976158142, 0...",0.0


In [19]:
# normalicemos los datos de froma que solo tengamos predicciones binarias:
pdf = pd.get_dummies(pdf, columns=['SpeciesID'], drop_first = True)

# cambiemos el tipo de ddato para que se pueda trabajer con el mas adelante:
pdf = pdf.astype({'SpeciesID_1.0': 'float', 'SpeciesID_2.0': 'float'})

In [20]:
# mostremos el resultado:
pdf.dtypes

Id                 int32
SepalLengthCm    float32
SepalWidthCm     float32
PetalLengthCm    float32
PetalWidthCm     float32
Species           object
Sizes             object
SpeciesID_1.0    float64
SpeciesID_2.0    float64
dtype: object

Una vez hemos llegado a este punto, crearemos dos particiones del modelo, para ello:

In [21]:
#reconvertimos el data frmae a spark data frame:
sdf = spark.createDataFrame(pdf)
sdf_col=sdf.columns
sdf.dtypes

[('Id', 'bigint'),
 ('SepalLengthCm', 'double'),
 ('SepalWidthCm', 'double'),
 ('PetalLengthCm', 'double'),
 ('PetalWidthCm', 'double'),
 ('Species', 'string'),
 ('Sizes', 'vector'),
 ('SpeciesID_1.0', 'double'),
 ('SpeciesID_2.0', 'double')]

AnalysisException: cannot resolve '`SpeciesID_1.0`' given input columns: [Id, PetalLengthCm, PetalWidthCm, SepalLengthCm, SepalWidthCm, Sizes, Species, SpeciesID];
'Project [Sizes#501, unresolvedalias(cast('SpeciesID_1.0 as int), None), unresolvedalias(cast('SpeciesID_2.0 as int), None)]
+- Project [Id#340, SepalLengthCm#341, SepalWidthCm#342, PetalLengthCm#343, PetalWidthCm#344, Species#21, Sizes#501, UDF(cast(Species#21 as string)) AS SpeciesID#806]
   +- Project [Id#340, SepalLengthCm#341, SepalWidthCm#342, PetalLengthCm#343, PetalWidthCm#344, Species#21, UDF(struct(SepalLengthCm_double_VectorAssembler_ea7370da9b12, cast(SepalLengthCm#341 as double), SepalWidthCm_double_VectorAssembler_ea7370da9b12, cast(SepalWidthCm#342 as double), PetalLengthCm_double_VectorAssembler_ea7370da9b12, cast(PetalLengthCm#343 as double), PetalWidthCm_double_VectorAssembler_ea7370da9b12, cast(PetalWidthCm#344 as double))) AS Sizes#501]
      +- Project [cast(Id#16 as int) AS Id#340, cast(SepalLengthCm#17

In [22]:
#creemos un nuevo vector con sizes y speciesID_1.0 y con ello un nuevo data set
req = 
assembler = VectorAssembler(inputCols=req, outputCol='Sizes+')

# añadamoslo al dataset y eliminemos las colunas que sobran
sdf_2 = assembler.transform(sdf).drop('Sizes', sdf_col[-2])

In [23]:
# primero crearemos la particion prescindiendo de la columna 'SpeciesID_2.0' para realizar ña primera predicción
(train_1, test_1) = sdf.drop(sdf_col[-1]).randomSplit([0.85, 0.15])

# Segundo, crearemos la particion para la segunda prediccion:
(train_2, test_2) = sdf_2.randomSplit([0.85, 0.15])

In [24]:
# Creemos el modelo de la primera prediccion:
gbtc_1 = GBTC(labelCol=sdf_col[-2], featuresCol='Sizes', maxIter=10)  # labelCol == SpeciesID_1.0

In [ ]:
# Entrenemos el modelo de la primera predicción:
gbtc_model_1 = gbtc_1.fit(train_1)

**no entiendo porque pero no me deja hacer la prediccion, continuare como si me dejara** 

In [ ]:
# una vez creado el modelo, hagamos la prediccion para la especie Versicolor:
gbtc_pred_1 = gbtc_model_1.transform(test_1)

Una vez tenemos la prediccion para Versicolor, haremos para Virginica, pero la haremos de un modo especial:

In [ ]:
# adaptamos la prediccion obtenida a nuestras necesidades:
req = sdf_col[-3:-2]
assembler = VectorAssembler(inputCols=req, outputCol='Sizes+')

# añadamoslo al dataset y eliminemos las colunas que sobran
sdf_2 = assembler.transform(sdf).drop('Sizes', sdf_col[-2])

In [ ]:
#crearemos un nuevo modelo predictivo empleando columnas del segundo data set:
gbtc_2 = GBTC(labelCol=sdf_col[-1], featuresCol='Sizes+', maxIter=10) # labelCol == SpeciesID_2.0

In [ ]:
# entrenaremos el modelo empleando test_2 obtenido de sdf_2:
gbtc_model_2 = gbtc_2.fit(train_2)

In [ ]:
# Ahora en vez de realizar la prediccion sobre 

### Bibliografia:
   * Páfginas Web:
      * https://sparkbyexamples.com/pyspark/convert-pandas-to-pyspark-dataframe/
   * Otros:
       * Ejercicio 5 de la asignatura Fundamento de Big Data